In [17]:
import urllib
import pandas
import json
import logging
import requests

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

def handler(event, context):
    
    #PROD:
#     xl_url = json.loads(event['body'])['url']
#     processed_url = json.loads(event['body'])['uploadUrl']
#     processed_file_name = json.loads(event['body'])['processedFileName']
#     original_xl = urllib.request.urlopen(xl_url)
#     df = pandas.read_excel(original_xl, skiprows=[0])
    
#     TESTING:
    df = pandas.read_excel("/Volumes/GoogleDrive/My Drive/Enterprise digital/Clients/QIC/Swift Engagement 2019/PwC Working Files/2020 Files/MassImports2020ProjectLevelNEW.xlsx", skiprows=[0])
    
    #Filter unnecessary asset categories
    assetCategories = ["02 Land", "03 Building", "08 Project Pools", "04 Plant and Equipment"]
    df = df[df.Classification.isin(assetCategories)]
    
    # Columns to retain/rename from IO export
    colsToRetain = ['Amount',
                    'Description', #rename to 'Asset Description'
                    'Classification', #Depreciate'
                    'Sub classification 1',
                    'Sub classification 2',
                    'Entity',
                    'Cost_centre',
                    'Date',
                    'Natural_account'
                   ]

    # Retain above and delete the rest
    df.drop(df.columns.difference(colsToRetain), 1, inplace=True)

    # Rename columns for ERP
    df.rename(columns={'Description':'*Asset Description',
                       'Amount':'*Cost',
                       'Classification': 'Asset Category Segment1',
                       'Sub classification 1': 'Asset Category Segment2',
                       'Sub classification 2': 'Asset Category Segment3',
                       'Cost_centre': 'Cost Centre',
                       'Natural_account': 'Natural Account'
                      },inplace=True)

    # Columns to add    
    colsToAdd = [#'*Interface Line Number',
                '*Asset Book',
                'Transaction Name',
                'Asset Number',
                #'*Asset Description',
                'Tag Number',
                'Manufacturer',
                'Serial Number',
                'Model',
                'Asset Type',
                #'*Cost',
                'Date Placed in Service',
                'Prorate Convention',
                '*Asset Units',
                #'Asset Category Segment1',
                #'Asset Category Segment2',
                #'Asset Category Segment3',
                'Asset Category Segment4',
                'Asset Category Segment5',
                'Asset Category Segment6',
                'Asset Category Segment7',
                '*Posting Status',
                'Queue Name',
                'Feeder System',
                'Parent Asset',
                'Add to Asset',
                'Asset Key Segment1',
                'Asset Key Segment2',
                'Asset Key Segment3',
                'Asset Key Segment4',
                'Asset Key Segment5',
                'Asset Key Segment6',
                'Asset Key Segment7',
                'Asset Key Segment8',
                'Asset Key Segment9',
                'Asset Key Segment10',
                'In physical inventory',
                'Property Type',
                'Property Class',
                'In use',
                'Ownership',
                'Bought',
                'Material Indicator',
                'Commitment',
                'Investment Law',
                'Amortize',
                'Amortization Start Date',
                '*Depreciate',
                'Salvage Value Type',
                'Salvage Value Amount',
                'Salvage Value Percent',
                'YTD Depreciation',
                'Depreciation Reserve',
                'YTD Bonus Depreciation',
                'Bonus Depreciation Reserve',
                'YTD Impairment',
                'Impairment Reserve',
                'Depreciation Method',
                'Life in Months',
                'Basic Rate',
                'Adjusted Rate',
                'Unit of Measure',
                'Production Capacity',
                'Ceiling Type',
                'Bonus Rule',
                'Cash Generating Unit',
                'Depreciation Limit Type',
                'Depreciation Limit Percent',
                'Depreciation Limit Amount',
                'Invoice Cost',
                'Cost Clearing Account Segment1',
                'Cost Clearing Account Segment2',
                'Cost Clearing Account Segment3',
                'Cost Clearing Account Segment4',
                'Cost Clearing Account Segment5',
                'Cost Clearing Account Segment6',
                'Cost Clearing Account Segment7',
                'Cost Clearing Account Segment8',
                'Cost Clearing Account Segment9',
                'Cost Clearing Account Segment10',
                'Cost Clearing Account Segment11',
                'Cost Clearing Account Segment12',
                'Cost Clearing Account Segment13',
                'Cost Clearing Account Segment14',
                'Cost Clearing Account Segment15',
                'Cost Clearing Account Segment16',
                'Cost Clearing Account Segment17',
                'Cost Clearing Account Segment18',
                'Cost Clearing Account Segment19',
                'Cost Clearing Account Segment20',
                'Cost Clearing Account Segment21',
                'Cost Clearing Account Segment22',
                'Cost Clearing Account Segment23',
                'Cost Clearing Account Segment24',
                'Cost Clearing Account Segment25',
                'Cost Clearing Account Segment26',
                'Cost Clearing Account Segment27',
                'Cost Clearing Account Segment28',
                'Cost Clearing Account Segment29',
                'Cost Clearing Account Segment30',
                'ATTRIBUTE1',
                'ATTRIBUTE2',
                'ATTRIBUTE3',
                'ATTRIBUTE4',
                'ATTRIBUTE5',
                'ATTRIBUTE6',
                'ATTRIBUTE7',
                'ATTRIBUTE8',
                'ATTRIBUTE9',
                'ATTRIBUTE10',
                'ATTRIBUTE11',
                'ATTRIBUTE12',
                'ATTRIBUTE13',
                'ATTRIBUTE14',
                'ATTRIBUTE15',
                'ATTRIBUTE16',
                'ATTRIBUTE17',
                'ATTRIBUTE18',
                'ATTRIBUTE19',
                'ATTRIBUTE20',
                'ATTRIBUTE21',
                'ATTRIBUTE22',
                'ATTRIBUTE23',
                'ATTRIBUTE24',
                'ATTRIBUTE25',
                'ATTRIBUTE26',
                'ATTRIBUTE27',
                'ATTRIBUTE28',
                'ATTRIBUTE29',
                'ATTRIBUTE30',
                'ATTRIBUTE_NUMBER1',
                'ATTRIBUTE_NUMBER2',
                'ATTRIBUTE_NUMBER3',
                'ATTRIBUTE_NUMBER4',
                'ATTRIBUTE_NUMBER5',
                'ATTRIBUTE_DATE1',
                'ATTRIBUTE_DATE2',
                'ATTRIBUTE_DATE3',
                'ATTRIBUTE_DATE4',
                'ATTRIBUTE_DATE5',
                'ATTRIBUTE_CATEGORY_CODE',
                'Context',
                'TH_ATTRIBUTE1',
                'TH_ATTRIBUTE2',
                'TH_ATTRIBUTE3',
                'TH_ATTRIBUTE4',
                'TH_ATTRIBUTE5',
                'TH_ATTRIBUTE6',
                'TH_ATTRIBUTE7',
                'TH_ATTRIBUTE8',
                'TH_ATTRIBUTE9',
                'TH_ATTRIBUTE10',
                'TH_ATTRIBUTE11',
                'TH_ATTRIBUTE12',
                'TH_ATTRIBUTE13',
                'TH_ATTRIBUTE14',
                'TH_ATTRIBUTE15',
                'TH_ATTRIBUTE_NUMBER1',
                'TH_ATTRIBUTE_NUMBER2',
                'TH_ATTRIBUTE_NUMBER3',
                'TH_ATTRIBUTE_NUMBER4',
                'TH_ATTRIBUTE_NUMBER5',
                'TH_ATTRIBUTE_DATE1',
                'TH_ATTRIBUTE_DATE2',
                'TH_ATTRIBUTE_DATE3',
                'TH_ATTRIBUTE_DATE4',
                'TH_ATTRIBUTE_DATE5',
                'TH_ATTRIBUTE_CATEGORY_CODE',
                'TH2_ATTRIBUTE1',
                'TH2_ATTRIBUTE2',
                'TH2_ATTRIBUTE3',
                'TH2_ATTRIBUTE4',
                'TH2_ATTRIBUTE5',
                'TH2_ATTRIBUTE6',
                'TH2_ATTRIBUTE7',
                'TH2_ATTRIBUTE8',
                'TH2_ATTRIBUTE9',
                'TH2_ATTRIBUTE10',
                'TH2_ATTRIBUTE11',
                'TH2_ATTRIBUTE12',
                'TH2_ATTRIBUTE13',
                'TH2_ATTRIBUTE14',
                'TH2_ATTRIBUTE15',
                'TH2_ATTRIBUTE_NUMBER1',
                'TH2_ATTRIBUTE_NUMBER2',
                'TH2_ATTRIBUTE_NUMBER3',
                'TH2_ATTRIBUTE_NUMBER4',
                'TH2_ATTRIBUTE_NUMBER5',
                'TH2_ATTRIBUTE_DATE1',
                'TH2_ATTRIBUTE_DATE2',
                'TH2_ATTRIBUTE_DATE3',
                'TH2_ATTRIBUTE_DATE4',
                'TH2_ATTRIBUTE_DATE5',
                'TH2_ATTRIBUTE_CATEGORY_CODE',
                'AI_ATTRIBUTE1',
                'AI_ATTRIBUTE2',
                'AI_ATTRIBUTE3',
                'AI_ATTRIBUTE4',
                'AI_ATTRIBUTE5',
                'AI_ATTRIBUTE6',
                'AI_ATTRIBUTE7',
                'AI_ATTRIBUTE8',
                'AI_ATTRIBUTE9',
                'AI_ATTRIBUTE10',
                'AI_ATTRIBUTE11',
                'AI_ATTRIBUTE12',
                'AI_ATTRIBUTE13',
                'AI_ATTRIBUTE14',
                'AI_ATTRIBUTE15',
                'AI_ATTRIBUTE_NUMBER1',
                'AI_ATTRIBUTE_NUMBER2',
                'AI_ATTRIBUTE_NUMBER3',
                'AI_ATTRIBUTE_NUMBER4',
                'AI_ATTRIBUTE_NUMBER5',
                'AI_ATTRIBUTE_DATE1',
                'AI_ATTRIBUTE_DATE2',
                'AI_ATTRIBUTE_DATE3',
                'AI_ATTRIBUTE_DATE4',
                'AI_ATTRIBUTE_DATE5',
                'AI_ATTRIBUTE_CATEGORY_CODE',
                'Mass Property Eligible',
                'Group Asset',
                'Reduction Rate',
                'Apply Reduction Rate to Additions',
                'Apply Reduction Rate to Adjustments',
                'Apply Reduction Rate to Retirements',
                'Recognize Gain or Loss',
                'Recapture Excess Reserve',
                'Limit Net Proceeds to Cost',
                'Terminal Gain or Loss',
                'Tracking Method',
                'Allocate Excess Depreciation',
                'Depreciate By',
                'Member Rollup',
                'Allocate to Fully Retired and Reserved Assets',
                'Over Depreciate',
                'Preparer',
                'Merged Level',
                'Parent Interface Line Number',
                'Sum Merged Units',
                'New Master',
                'Units to Adjust',
                'Short year',
                'Conversion Date',
                'Original Depreciation Start Date',
                'GLOBAL_ATTRIBUTE1',
                'GLOBAL_ATTRIBUTE2',
                'GLOBAL_ATTRIBUTE3',
                'GLOBAL_ATTRIBUTE4',
                'GLOBAL_ATTRIBUTE5',
                'GLOBAL_ATTRIBUTE6',
                'GLOBAL_ATTRIBUTE7',
                'GLOBAL_ATTRIBUTE8',
                'GLOBAL_ATTRIBUTE9',
                'GLOBAL_ATTRIBUTE10',
                'GLOBAL_ATTRIBUTE11',
                'GLOBAL_ATTRIBUTE12',
                'GLOBAL_ATTRIBUTE13',
                'GLOBAL_ATTRIBUTE14',
                'GLOBAL_ATTRIBUTE15',
                'GLOBAL_ATTRIBUTE16',
                'GLOBAL_ATTRIBUTE17',
                'GLOBAL_ATTRIBUTE18',
                'GLOBAL_ATTRIBUTE19',
                'GLOBAL_ATTRIBUTE20',
                'GLOBAL_ATTRIBUTE_NUMBER1',
                'GLOBAL_ATTRIBUTE_NUMBER2',
                'GLOBAL_ATTRIBUTE_NUMBER3',
                'GLOBAL_ATTRIBUTE_NUMBER4',
                'GLOBAL_ATTRIBUTE_NUMBER5',
                'GLOBAL_ATTRIBUTE_DATE1',
                'GLOBAL_ATTRIBUTE_DATE2',
                'GLOBAL_ATTRIBUTE_DATE3',
                'GLOBAL_ATTRIBUTE_DATE4',
                'GLOBAL_ATTRIBUTE_DATE5',
                'GLOBAL_ATTRIBUTE_CATEGORY',
                'Net Book Value at the Time of Switch',
                'Period Fully Reserved',
                'Start Period of Extended Depreciation',
                'Earlier Depreciation Limit Type',
                'Earlier Depreciation Limit Percent',
                'Earlier Depreciation Limit Amount',
                'Earlier Depreciation Method',
                'Earlier Life in Months',
                'Earlier Basic Rate',
                'Earlier Adjusted Rate',
                'Asset Schedule Identifier',
                'Lease Number',
                'Revaluation Reserve',
                'Revaluation Loss',
                'Revaluation Reserve Amortization Basis',
                'Impairment Loss Expense',
                'Revaluation Cost Ceiling',
                'Fair Value',
                'Last Used Price Index Value',
                'GLOBAL_ATTRIBUTE_NUMBER6',
                'GLOBAL_ATTRIBUTE_NUMBER7',
                'GLOBAL_ATTRIBUTE_NUMBER8',
                'GLOBAL_ATTRIBUTE_NUMBER9',
                'GLOBAL_ATTRIBUTE_NUMBER10',
                'GLOBAL_ATTRIBUTE_DATE6',
                'GLOBAL_ATTRIBUTE_DATE7',
                'GLOBAL_ATTRIBUTE_DATE8',
                'GLOBAL_ATTRIBUTE_DATE9',
                'GLOBAL_ATTRIBUTE_DATE10',
                'BK_GLOBAL_ATTRIBUTE1',
                'BK_GLOBAL_ATTRIBUTE2',
                'BK_GLOBAL_ATTRIBUTE3',
                'BK_GLOBAL_ATTRIBUTE4',
                'BK_GLOBAL_ATTRIBUTE5',
                'BK_GLOBAL_ATTRIBUTE6',
                'BK_GLOBAL_ATTRIBUTE7',
                'BK_GLOBAL_ATTRIBUTE8',
                'BK_GLOBAL_ATTRIBUTE9',
                'BK_GLOBAL_ATTRIBUTE10',
                'BK_GLOBAL_ATTRIBUTE11',
                'BK_GLOBAL_ATTRIBUTE12',
                'BK_GLOBAL_ATTRIBUTE13',
                'BK_GLOBAL_ATTRIBUTE14',
                'BK_GLOBAL_ATTRIBUTE15',
                'BK_GLOBAL_ATTRIBUTE16',
                'BK_GLOBAL_ATTRIBUTE17',
                'BK_GLOBAL_ATTRIBUTE18',
                'BK_GLOBAL_ATTRIBUTE19',
                'BK_GLOBAL_ATTRIBUTE20',
                'BK_GLOBAL_ATTRIBUTE_NUMBER1',
                'BK_GLOBAL_ATTRIBUTE_NUMBER2',
                'BK_GLOBAL_ATTRIBUTE_NUMBER3',
                'BK_GLOBAL_ATTRIBUTE_NUMBER4',
                'BK_GLOBAL_ATTRIBUTE_NUMBER5',
                'BK_GLOBAL_ATTRIBUTE_DATE1',
                'BK_GLOBAL_ATTRIBUTE_DATE2',
                'BK_GLOBAL_ATTRIBUTE_DATE3',
                'BK_GLOBAL_ATTRIBUTE_DATE4',
                'BK_GLOBAL_ATTRIBUTE_DATE5',
                'BK_GLOBAL_ATTRIBUTE_CATEGORY',
                'TH_GLOBAL_ATTRIBUTE1',
                'TH_GLOBAL_ATTRIBUTE2',
                'TH_GLOBAL_ATTRIBUTE3',
                'TH_GLOBAL_ATTRIBUTE4',
                'TH_GLOBAL_ATTRIBUTE5',
                'TH_GLOBAL_ATTRIBUTE6',
                'TH_GLOBAL_ATTRIBUTE7',
                'TH_GLOBAL_ATTRIBUTE8',
                'TH_GLOBAL_ATTRIBUTE9',
                'TH_GLOBAL_ATTRIBUTE10',
                'TH_GLOBAL_ATTRIBUTE11',
                'TH_GLOBAL_ATTRIBUTE12',
                'TH_GLOBAL_ATTRIBUTE13',
                'TH_GLOBAL_ATTRIBUTE14',
                'TH_GLOBAL_ATTRIBUTE15',
                'TH_GLOBAL_ATTRIBUTE16',
                'TH_GLOBAL_ATTRIBUTE17',
                'TH_GLOBAL_ATTRIBUTE18',
                'TH_GLOBAL_ATTRIBUTE19',
                'TH_GLOBAL_ATTRIBUTE20',
                'TH_GLOBAL_ATTRIBUTE_NUMBER1',
                'TH_GLOBAL_ATTRIBUTE_NUMBER2',
                'TH_GLOBAL_ATTRIBUTE_NUMBER3',
                'TH_GLOBAL_ATTRIBUTE_NUMBER4',
                'TH_GLOBAL_ATTRIBUTE_NUMBER5',
                'TH_GLOBAL_ATTRIBUTE_DATE1',
                'TH_GLOBAL_ATTRIBUTE_DATE2',
                'TH_GLOBAL_ATTRIBUTE_DATE3',
                'TH_GLOBAL_ATTRIBUTE_DATE4',
                'TH_GLOBAL_ATTRIBUTE_DATE5',
                'TH_GLOBAL_ATTRIBUTE_CATEGORY',
                'AI_GLOBAL_ATTRIBUTE1',
                'AI_GLOBAL_ATTRIBUTE2',
                'AI_GLOBAL_ATTRIBUTE3',
                'AI_GLOBAL_ATTRIBUTE4',
                'AI_GLOBAL_ATTRIBUTE5',
                'AI_GLOBAL_ATTRIBUTE6',
                'AI_GLOBAL_ATTRIBUTE7',
                'AI_GLOBAL_ATTRIBUTE8',
                'AI_GLOBAL_ATTRIBUTE9',
                'AI_GLOBAL_ATTRIBUTE10',
                'AI_GLOBAL_ATTRIBUTE11',
                'AI_GLOBAL_ATTRIBUTE12',
                'AI_GLOBAL_ATTRIBUTE13',
                'AI_GLOBAL_ATTRIBUTE14',
                'AI_GLOBAL_ATTRIBUTE15',
                'AI_GLOBAL_ATTRIBUTE16',
                'AI_GLOBAL_ATTRIBUTE17',
                'AI_GLOBAL_ATTRIBUTE18',
                'AI_GLOBAL_ATTRIBUTE19',
                'AI_GLOBAL_ATTRIBUTE20',
                'AI_GLOBAL_ATTRIBUTE_NUMBER1',
                'AI_GLOBAL_ATTRIBUTE_NUMBER2',
                'AI_GLOBAL_ATTRIBUTE_NUMBER3',
                'AI_GLOBAL_ATTRIBUTE_NUMBER4',
                'AI_GLOBAL_ATTRIBUTE_NUMBER5',
                'AI_GLOBAL_ATTRIBUTE_DATE1',
                'AI_GLOBAL_ATTRIBUTE_DATE2',
                'AI_GLOBAL_ATTRIBUTE_DATE3',
                'AI_GLOBAL_ATTRIBUTE_DATE4',
                'AI_GLOBAL_ATTRIBUTE_DATE5',
                'AI_GLOBAL_ATTRIBUTE_CATEGORY',
                'Supplier Name',
                'Supplier Number',
                'Purchase Order Number',
                'Invoice Number',
                'Invoice Voucher Number',
                'Invoice Date',
                'Payables Units',
                'Invoice Line Number',
                'Invoice Line Type',
                'Invoice Line Description',
                'Invoice Payment Number',
                'Project Number',
                'Task Number',
                'Fully depreciate',
                'Depreciation Factor', 
                ]


    # Add new columns
    df[colsToAdd] = pandas.DataFrame([len(colsToAdd)*['']], index=df.index)
    df.insert(0,'*Interface Line Number',df.index)
    df['*Interface Line Number'] += 1

    # Update columns
    df['*Asset Book']='OPS CORP'
    df['*Asset Units']= 1
   
    
    # Rearrange Columns
    cols = ['Entity',
            'Cost Centre',
            'Natural Account',
            'Date',
            '*Interface Line Number',
            '*Asset Book',
            'Transaction Name',
            'Asset Number',
            '*Asset Description',
            'Tag Number',
            'Manufacturer',
            'Serial Number',
            'Model',
            'Asset Type',
            '*Cost',
            'Date Placed in Service',
            'Prorate Convention',
            '*Asset Units',
            'Asset Category Segment1',
            'Asset Category Segment2',
            'Asset Category Segment3',
            'Asset Category Segment4',
            'Asset Category Segment5',
            'Asset Category Segment6',
            'Asset Category Segment7',
            '*Posting Status',
            'Queue Name',
            'Feeder System',
            'Parent Asset',
            'Add to Asset',
            'Asset Key Segment1',
            'Asset Key Segment2',
            'Asset Key Segment3',
            'Asset Key Segment4',
            'Asset Key Segment5',
            'Asset Key Segment6',
            'Asset Key Segment7',
            'Asset Key Segment8',
            'Asset Key Segment9',
            'Asset Key Segment10',
            'In physical inventory',
            'Property Type',
            'Property Class',
            'In use',
            'Ownership',
            'Bought',
            'Material Indicator',
            'Commitment',
            'Investment Law',
            'Amortize',
            'Amortization Start Date',
            '*Depreciate',
            'Salvage Value Type',
            'Salvage Value Amount',
            'Salvage Value Percent',
            'YTD Depreciation',
            'Depreciation Reserve',
            'YTD Bonus Depreciation',
            'Bonus Depreciation Reserve',
            'YTD Impairment',
            'Impairment Reserve',
            'Depreciation Method',
            'Life in Months',
            'Basic Rate',
            'Adjusted Rate',
            'Unit of Measure',
            'Production Capacity',
            'Ceiling Type',
            'Bonus Rule',
            'Cash Generating Unit',
            'Depreciation Limit Type',
            'Depreciation Limit Percent',
            'Depreciation Limit Amount',
            'Invoice Cost',
            'Cost Clearing Account Segment1',
            'Cost Clearing Account Segment2',
            'Cost Clearing Account Segment3',
            'Cost Clearing Account Segment4',
            'Cost Clearing Account Segment5',
            'Cost Clearing Account Segment6',
            'Cost Clearing Account Segment7',
            'Cost Clearing Account Segment8',
            'Cost Clearing Account Segment9',
            'Cost Clearing Account Segment10',
            'Cost Clearing Account Segment11',
            'Cost Clearing Account Segment12',
            'Cost Clearing Account Segment13',
            'Cost Clearing Account Segment14',
            'Cost Clearing Account Segment15',
            'Cost Clearing Account Segment16',
            'Cost Clearing Account Segment17',
            'Cost Clearing Account Segment18',
            'Cost Clearing Account Segment19',
            'Cost Clearing Account Segment20',
            'Cost Clearing Account Segment21',
            'Cost Clearing Account Segment22',
            'Cost Clearing Account Segment23',
            'Cost Clearing Account Segment24',
            'Cost Clearing Account Segment25',
            'Cost Clearing Account Segment26',
            'Cost Clearing Account Segment27',
            'Cost Clearing Account Segment28',
            'Cost Clearing Account Segment29',
            'Cost Clearing Account Segment30',
            'ATTRIBUTE1',
            'ATTRIBUTE2',
            'ATTRIBUTE3',
            'ATTRIBUTE4',
            'ATTRIBUTE5',
            'ATTRIBUTE6',
            'ATTRIBUTE7',
            'ATTRIBUTE8',
            'ATTRIBUTE9',
            'ATTRIBUTE10',
            'ATTRIBUTE11',
            'ATTRIBUTE12',
            'ATTRIBUTE13',
            'ATTRIBUTE14',
            'ATTRIBUTE15',
            'ATTRIBUTE16',
            'ATTRIBUTE17',
            'ATTRIBUTE18',
            'ATTRIBUTE19',
            'ATTRIBUTE20',
            'ATTRIBUTE21',
            'ATTRIBUTE22',
            'ATTRIBUTE23',
            'ATTRIBUTE24',
            'ATTRIBUTE25',
            'ATTRIBUTE26',
            'ATTRIBUTE27',
            'ATTRIBUTE28',
            'ATTRIBUTE29',
            'ATTRIBUTE30',
            'ATTRIBUTE_NUMBER1',
            'ATTRIBUTE_NUMBER2',
            'ATTRIBUTE_NUMBER3',
            'ATTRIBUTE_NUMBER4',
            'ATTRIBUTE_NUMBER5',
            'ATTRIBUTE_DATE1',
            'ATTRIBUTE_DATE2',
            'ATTRIBUTE_DATE3',
            'ATTRIBUTE_DATE4',
            'ATTRIBUTE_DATE5',
            'ATTRIBUTE_CATEGORY_CODE',
            'Context',
            'TH_ATTRIBUTE1',
            'TH_ATTRIBUTE2',
            'TH_ATTRIBUTE3',
            'TH_ATTRIBUTE4',
            'TH_ATTRIBUTE5',
            'TH_ATTRIBUTE6',
            'TH_ATTRIBUTE7',
            'TH_ATTRIBUTE8',
            'TH_ATTRIBUTE9',
            'TH_ATTRIBUTE10',
            'TH_ATTRIBUTE11',
            'TH_ATTRIBUTE12',
            'TH_ATTRIBUTE13',
            'TH_ATTRIBUTE14',
            'TH_ATTRIBUTE15',
            'TH_ATTRIBUTE_NUMBER1',
            'TH_ATTRIBUTE_NUMBER2',
            'TH_ATTRIBUTE_NUMBER3',
            'TH_ATTRIBUTE_NUMBER4',
            'TH_ATTRIBUTE_NUMBER5',
            'TH_ATTRIBUTE_DATE1',
            'TH_ATTRIBUTE_DATE2',
            'TH_ATTRIBUTE_DATE3',
            'TH_ATTRIBUTE_DATE4',
            'TH_ATTRIBUTE_DATE5',
            'TH_ATTRIBUTE_CATEGORY_CODE',
            'TH2_ATTRIBUTE1',
            'TH2_ATTRIBUTE2',
            'TH2_ATTRIBUTE3',
            'TH2_ATTRIBUTE4',
            'TH2_ATTRIBUTE5',
            'TH2_ATTRIBUTE6',
            'TH2_ATTRIBUTE7',
            'TH2_ATTRIBUTE8',
            'TH2_ATTRIBUTE9',
            'TH2_ATTRIBUTE10',
            'TH2_ATTRIBUTE11',
            'TH2_ATTRIBUTE12',
            'TH2_ATTRIBUTE13',
            'TH2_ATTRIBUTE14',
            'TH2_ATTRIBUTE15',
            'TH2_ATTRIBUTE_NUMBER1',
            'TH2_ATTRIBUTE_NUMBER2',
            'TH2_ATTRIBUTE_NUMBER3',
            'TH2_ATTRIBUTE_NUMBER4',
            'TH2_ATTRIBUTE_NUMBER5',
            'TH2_ATTRIBUTE_DATE1',
            'TH2_ATTRIBUTE_DATE2',
            'TH2_ATTRIBUTE_DATE3',
            'TH2_ATTRIBUTE_DATE4',
            'TH2_ATTRIBUTE_DATE5',
            'TH2_ATTRIBUTE_CATEGORY_CODE',
            'AI_ATTRIBUTE1',
            'AI_ATTRIBUTE2',
            'AI_ATTRIBUTE3',
            'AI_ATTRIBUTE4',
            'AI_ATTRIBUTE5',
            'AI_ATTRIBUTE6',
            'AI_ATTRIBUTE7',
            'AI_ATTRIBUTE8',
            'AI_ATTRIBUTE9',
            'AI_ATTRIBUTE10',
            'AI_ATTRIBUTE11',
            'AI_ATTRIBUTE12',
            'AI_ATTRIBUTE13',
            'AI_ATTRIBUTE14',
            'AI_ATTRIBUTE15',
            'AI_ATTRIBUTE_NUMBER1',
            'AI_ATTRIBUTE_NUMBER2',
            'AI_ATTRIBUTE_NUMBER3',
            'AI_ATTRIBUTE_NUMBER4',
            'AI_ATTRIBUTE_NUMBER5',
            'AI_ATTRIBUTE_DATE1',
            'AI_ATTRIBUTE_DATE2',
            'AI_ATTRIBUTE_DATE3',
            'AI_ATTRIBUTE_DATE4',
            'AI_ATTRIBUTE_DATE5',
            'AI_ATTRIBUTE_CATEGORY_CODE',
            'Mass Property Eligible',
            'Group Asset',
            'Reduction Rate',
            'Apply Reduction Rate to Additions',
            'Apply Reduction Rate to Adjustments',
            'Apply Reduction Rate to Retirements',
            'Recognize Gain or Loss',
            'Recapture Excess Reserve',
            'Limit Net Proceeds to Cost',
            'Terminal Gain or Loss',
            'Tracking Method',
            'Allocate Excess Depreciation',
            'Depreciate By',
            'Member Rollup',
            'Allocate to Fully Retired and Reserved Assets',
            'Over Depreciate',
            'Preparer',
            'Merged Level',
            'Parent Interface Line Number',
            'Sum Merged Units',
            'New Master',
            'Units to Adjust',
            'Short year',
            'Conversion Date',
            'Original Depreciation Start Date',
            'GLOBAL_ATTRIBUTE1',
            'GLOBAL_ATTRIBUTE2',
            'GLOBAL_ATTRIBUTE3',
            'GLOBAL_ATTRIBUTE4',
            'GLOBAL_ATTRIBUTE5',
            'GLOBAL_ATTRIBUTE6',
            'GLOBAL_ATTRIBUTE7',
            'GLOBAL_ATTRIBUTE8',
            'GLOBAL_ATTRIBUTE9',
            'GLOBAL_ATTRIBUTE10',
            'GLOBAL_ATTRIBUTE11',
            'GLOBAL_ATTRIBUTE12',
            'GLOBAL_ATTRIBUTE13',
            'GLOBAL_ATTRIBUTE14',
            'GLOBAL_ATTRIBUTE15',
            'GLOBAL_ATTRIBUTE16',
            'GLOBAL_ATTRIBUTE17',
            'GLOBAL_ATTRIBUTE18',
            'GLOBAL_ATTRIBUTE19',
            'GLOBAL_ATTRIBUTE20',
            'GLOBAL_ATTRIBUTE_NUMBER1',
            'GLOBAL_ATTRIBUTE_NUMBER2',
            'GLOBAL_ATTRIBUTE_NUMBER3',
            'GLOBAL_ATTRIBUTE_NUMBER4',
            'GLOBAL_ATTRIBUTE_NUMBER5',
            'GLOBAL_ATTRIBUTE_DATE1',
            'GLOBAL_ATTRIBUTE_DATE2',
            'GLOBAL_ATTRIBUTE_DATE3',
            'GLOBAL_ATTRIBUTE_DATE4',
            'GLOBAL_ATTRIBUTE_DATE5',
            'GLOBAL_ATTRIBUTE_CATEGORY',
            'Net Book Value at the Time of Switch',
            'Period Fully Reserved',
            'Start Period of Extended Depreciation',
            'Earlier Depreciation Limit Type',
            'Earlier Depreciation Limit Percent',
            'Earlier Depreciation Limit Amount',
            'Earlier Depreciation Method',
            'Earlier Life in Months',
            'Earlier Basic Rate',
            'Earlier Adjusted Rate',
            'Asset Schedule Identifier',
            'Lease Number',
            'Revaluation Reserve',
            'Revaluation Loss',
            'Revaluation Reserve Amortization Basis',
            'Impairment Loss Expense',
            'Revaluation Cost Ceiling',
            'Fair Value',
            'Last Used Price Index Value',
            'GLOBAL_ATTRIBUTE_NUMBER6',
            'GLOBAL_ATTRIBUTE_NUMBER7',
            'GLOBAL_ATTRIBUTE_NUMBER8',
            'GLOBAL_ATTRIBUTE_NUMBER9',
            'GLOBAL_ATTRIBUTE_NUMBER10',
            'GLOBAL_ATTRIBUTE_DATE6',
            'GLOBAL_ATTRIBUTE_DATE7',
            'GLOBAL_ATTRIBUTE_DATE8',
            'GLOBAL_ATTRIBUTE_DATE9',
            'GLOBAL_ATTRIBUTE_DATE10',
            'BK_GLOBAL_ATTRIBUTE1',
            'BK_GLOBAL_ATTRIBUTE2',
            'BK_GLOBAL_ATTRIBUTE3',
            'BK_GLOBAL_ATTRIBUTE4',
            'BK_GLOBAL_ATTRIBUTE5',
            'BK_GLOBAL_ATTRIBUTE6',
            'BK_GLOBAL_ATTRIBUTE7',
            'BK_GLOBAL_ATTRIBUTE8',
            'BK_GLOBAL_ATTRIBUTE9',
            'BK_GLOBAL_ATTRIBUTE10',
            'BK_GLOBAL_ATTRIBUTE11',
            'BK_GLOBAL_ATTRIBUTE12',
            'BK_GLOBAL_ATTRIBUTE13',
            'BK_GLOBAL_ATTRIBUTE14',
            'BK_GLOBAL_ATTRIBUTE15',
            'BK_GLOBAL_ATTRIBUTE16',
            'BK_GLOBAL_ATTRIBUTE17',
            'BK_GLOBAL_ATTRIBUTE18',
            'BK_GLOBAL_ATTRIBUTE19',
            'BK_GLOBAL_ATTRIBUTE20',
            'BK_GLOBAL_ATTRIBUTE_NUMBER1',
            'BK_GLOBAL_ATTRIBUTE_NUMBER2',
            'BK_GLOBAL_ATTRIBUTE_NUMBER3',
            'BK_GLOBAL_ATTRIBUTE_NUMBER4',
            'BK_GLOBAL_ATTRIBUTE_NUMBER5',
            'BK_GLOBAL_ATTRIBUTE_DATE1',
            'BK_GLOBAL_ATTRIBUTE_DATE2',
            'BK_GLOBAL_ATTRIBUTE_DATE3',
            'BK_GLOBAL_ATTRIBUTE_DATE4',
            'BK_GLOBAL_ATTRIBUTE_DATE5',
            'BK_GLOBAL_ATTRIBUTE_CATEGORY',
            'TH_GLOBAL_ATTRIBUTE1',
            'TH_GLOBAL_ATTRIBUTE2',
            'TH_GLOBAL_ATTRIBUTE3',
            'TH_GLOBAL_ATTRIBUTE4',
            'TH_GLOBAL_ATTRIBUTE5',
            'TH_GLOBAL_ATTRIBUTE6',
            'TH_GLOBAL_ATTRIBUTE7',
            'TH_GLOBAL_ATTRIBUTE8',
            'TH_GLOBAL_ATTRIBUTE9',
            'TH_GLOBAL_ATTRIBUTE10',
            'TH_GLOBAL_ATTRIBUTE11',
            'TH_GLOBAL_ATTRIBUTE12',
            'TH_GLOBAL_ATTRIBUTE13',
            'TH_GLOBAL_ATTRIBUTE14',
            'TH_GLOBAL_ATTRIBUTE15',
            'TH_GLOBAL_ATTRIBUTE16',
            'TH_GLOBAL_ATTRIBUTE17',
            'TH_GLOBAL_ATTRIBUTE18',
            'TH_GLOBAL_ATTRIBUTE19',
            'TH_GLOBAL_ATTRIBUTE20',
            'TH_GLOBAL_ATTRIBUTE_NUMBER1',
            'TH_GLOBAL_ATTRIBUTE_NUMBER2',
            'TH_GLOBAL_ATTRIBUTE_NUMBER3',
            'TH_GLOBAL_ATTRIBUTE_NUMBER4',
            'TH_GLOBAL_ATTRIBUTE_NUMBER5',
            'TH_GLOBAL_ATTRIBUTE_DATE1',
            'TH_GLOBAL_ATTRIBUTE_DATE2',
            'TH_GLOBAL_ATTRIBUTE_DATE3',
            'TH_GLOBAL_ATTRIBUTE_DATE4',
            'TH_GLOBAL_ATTRIBUTE_DATE5',
            'TH_GLOBAL_ATTRIBUTE_CATEGORY',
            'AI_GLOBAL_ATTRIBUTE1',
            'AI_GLOBAL_ATTRIBUTE2',
            'AI_GLOBAL_ATTRIBUTE3',
            'AI_GLOBAL_ATTRIBUTE4',
            'AI_GLOBAL_ATTRIBUTE5',
            'AI_GLOBAL_ATTRIBUTE6',
            'AI_GLOBAL_ATTRIBUTE7',
            'AI_GLOBAL_ATTRIBUTE8',
            'AI_GLOBAL_ATTRIBUTE9',
            'AI_GLOBAL_ATTRIBUTE10',
            'AI_GLOBAL_ATTRIBUTE11',
            'AI_GLOBAL_ATTRIBUTE12',
            'AI_GLOBAL_ATTRIBUTE13',
            'AI_GLOBAL_ATTRIBUTE14',
            'AI_GLOBAL_ATTRIBUTE15',
            'AI_GLOBAL_ATTRIBUTE16',
            'AI_GLOBAL_ATTRIBUTE17',
            'AI_GLOBAL_ATTRIBUTE18',
            'AI_GLOBAL_ATTRIBUTE19',
            'AI_GLOBAL_ATTRIBUTE20',
            'AI_GLOBAL_ATTRIBUTE_NUMBER1',
            'AI_GLOBAL_ATTRIBUTE_NUMBER2',
            'AI_GLOBAL_ATTRIBUTE_NUMBER3',
            'AI_GLOBAL_ATTRIBUTE_NUMBER4',
            'AI_GLOBAL_ATTRIBUTE_NUMBER5',
            'AI_GLOBAL_ATTRIBUTE_DATE1',
            'AI_GLOBAL_ATTRIBUTE_DATE2',
            'AI_GLOBAL_ATTRIBUTE_DATE3',
            'AI_GLOBAL_ATTRIBUTE_DATE4',
            'AI_GLOBAL_ATTRIBUTE_DATE5',
            'AI_GLOBAL_ATTRIBUTE_CATEGORY',
            'Supplier Name',
            'Supplier Number',
            'Purchase Order Number',
            'Invoice Number',
            'Invoice Voucher Number',
            'Invoice Date',
            'Payables Units',
            'Invoice Line Number',
            'Invoice Line Type',
            'Invoice Line Description',
            'Invoice Payment Number',
            'Project Number',
            'Task Number',
            'Fully depreciate',
            'Depreciation Factor', 
            ]
    
    df = df[cols]

    # Clear NAs
    df = df.fillna(value='')
    
    df['Date'] = df['Date'].str[:10]

    
#     #PROD:
#     file_name = '/tmp/' + processed_file_name
#     df.to_csv(file_name, index=False)
#     r = requests.put(processed_url, data=open(file_name, 'rb'))
#     logger.info(r.status_code)
    
#     TESTING:
    df.to_csv("/Volumes/GoogleDrive/My Drive/Enterprise digital/Clients/QIC/Swift Engagement 2019/PwC Working Files/2020 Files/MassImportsOutput.csv")
    
    return {
        'statusCode': 200,
        'body': json.dumps('success'), #if r.status_code == 200 else json.dumps('failed'),
        'headers': {
            'Content-Type': 'application/json',
        },
    }


In [18]:
handler("1","1")

{'statusCode': 200,
 'body': '"success"',
 'headers': {'Content-Type': 'application/json'}}